In [ ]:
!wget https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
!unzip jena_climate_2009_2016.csv.zip

--2022-08-19 18:27:53--  https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.83.150
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.83.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13565642 (13M) [application/zip]
Saving to: ‘jena_climate_2009_2016.csv.zip’

jena_climate_2009_2 100%[===================>]  12.94M  5.12MB/s    in 2.5s    

2022-08-19 18:27:56 (5.12 MB/s) - ‘jena_climate_2009_2016.csv.zip’ saved [13565642/13565642]

Archive:  jena_climate_2009_2016.csv.zip
  inflating: jena_climate_2009_2016.csv  
  inflating: __MACOSX/._jena_climate_2009_2016.csv  


In [ ]:
import os
fname = os.path.join("jena_climate_2009_2016.csv")
with open(fname) as f:

  data = f.read()
lines = data.split("\n")
header = lines[0].split(",")
lines = lines[1:]
print(header)
print(len(lines))

['"Date Time"', '"p (mbar)"', '"T (degC)"', '"Tpot (K)"', '"Tdew (degC)"', '"rh (%)"', '"VPmax (mbar)"', '"VPact (mbar)"', '"VPdef (mbar)"', '"sh (g/kg)"', '"H2OC (mmol/mol)"', '"rho (g/m**3)"', '"wv (m/s)"', '"max. wv (m/s)"', '"wd (deg)"']
420451


In [ ]:
import numpy as np
temperature = np.zeros((len(lines),))
raw_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
  values = [float(x) for x in line.split(",")[1:]]
  temperature[i] = values[1]
  raw_data[i, :] = values[:]

In [ ]:
temperature

array([-8.02, -8.41, -8.51, ..., -3.16, -4.23, -4.82])

In [ ]:
num_train_samples = int(0.5 * len(raw_data))
num_val_samples = int(0.25 * len(raw_data))
num_test_samples = len(raw_data) - num_train_samples - num_val_samples
print("num_train_samples:", num_train_samples)
print("num_val_samples:", num_val_samples)
print("num_test_samples:", num_test_samples)

num_train_samples: 210225
num_val_samples: 105112
num_test_samples: 105114


In [ ]:
mean = raw_data[:num_train_samples].mean(axis=0)
raw_data -= mean
std = raw_data[:num_train_samples].std(axis=0)
raw_data /= std

In [ ]:
from tensorflow import keras
sampling_rate = 6

sequence_length = 120

delay = sampling_rate * (sequence_length + 24 - 1)

batch_size = 256

train_dataset = keras.utils.timeseries_dataset_from_array(
raw_data[:-delay],
targets=temperature[delay:],
sampling_rate=sampling_rate,
sequence_length=sequence_length,
shuffle=True,
batch_size=batch_size,
start_index=0,
end_index=num_train_samples)

val_dataset = keras.utils.timeseries_dataset_from_array(
raw_data[:-delay],
targets=temperature[delay:],
sampling_rate=sampling_rate,
sequence_length=sequence_length,
shuffle=True,
batch_size=batch_size,
start_index=num_train_samples,
end_index=num_train_samples + num_val_samples)
test_dataset = keras.utils.timeseries_dataset_from_array(
raw_data[:-delay],
targets=temperature[delay:],
sampling_rate=sampling_rate,
sequence_length=sequence_length,
shuffle=True,
batch_size=batch_size,
start_index=num_train_samples + num_val_samples)

In [ ]:
for samples, targets in train_dataset:
  print("samples shape:", samples.shape)
  print("targets shape:", targets.shape)
  break



samples shape: (256, 120, 14)
targets shape: (256,)


In [ ]:
def evaluate_naive_method(dataset):
  total_abs_err = 0.
  samples_seen = 0
  for samples, targets in dataset:
    preds = samples[:, -1, 1] * std[1] + mean[1]
    total_abs_err += np.sum(np.abs(preds - targets))
    samples_seen += samples.shape[0]
  return total_abs_err / samples_seen
print(f"Validation MAE: {evaluate_naive_method(val_dataset):.2f}")
print(f"Test MAE: {evaluate_naive_method(test_dataset):.2f}")

Validation MAE: 2.44
Test MAE: 2.62


In [ ]:
raw_data.shape[-1]

14

In [ ]:
raw_data.shape

(420451, 14)

simple layer with dense layer 


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.Flatten()(inputs)
x = layers.Dense(16, activation="relu")(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)
callbacks = [
keras.callbacks.ModelCheckpoint("jena_dense.keras",
save_best_only=True)
]


model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
epochs=10,
validation_data=val_dataset,
callbacks=callbacks)
model = keras.models.load_model("jena_dense.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/10
819/819 [==============================] - 46s 51ms/step - loss: 13.3425 - mae: 2.8311 - val_loss: 10.3109 - val_mae: 2.5401
Epoch 2/10
819/819 [==============================] - 44s 53ms/step - loss: 9.6192 - mae: 2.4378 - val_loss: 14.0945 - val_mae: 2.9746
Epoch 3/10
819/819 [==============================] - 43s 53ms/step - loss: 8.8011 - mae: 2.3368 - val_loss: 12.0653 - val_mae: 2.7501
Epoch 4/10
819/819 [==============================] - 42s 51ms/step - loss: 8.2362 - mae: 2.2612 - val_loss: 10.2839 - val_mae: 2.5265
Epoch 5/10
819/819 [==============================] - 42s 51ms/step - loss: 7.8423 - mae: 2.2055 - val_loss: 12.2125 - val_mae: 2.7539
Epoch 6/10
819/819 [==============================] - 44s 54ms/step - loss: 7.5081 - mae: 2.1572 - val_loss: 13.9334 - val_mae: 2.9627
Epoch 7/10
819/819 [==============================] - 44s 53ms/step - loss: 7.2881 - mae: 2.1249 - val_loss: 10.9488 - val_mae: 2.6070
Epoch 8/10
819/819 [==============================] - 

simple recurrent layer

In [ ]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.Conv1D(8, 24, activation="relu")(inputs)
x = layers.MaxPooling1D(2)(x)
x = layers.Conv1D(8, 12, activation="relu")(x)
x = layers.MaxPooling1D(2)(x)
x = layers.Conv1D(8, 6, activation="relu")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)
callbacks = [
keras.callbacks.ModelCheckpoint("jena_conv.keras",
save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
epochs=5,
validation_data=val_dataset,
callbacks=callbacks)
model = keras.models.load_model("jena_conv.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/5
819/819 [==============================] - 45s 53ms/step - loss: nan - accuracy: 2.4820e-04 - val_loss: nan - val_accuracy: 2.9695e-04
Epoch 2/5
819/819 [==============================] - 44s 53ms/step - loss: nan - accuracy: 2.4820e-04 - val_loss: nan - val_accuracy: 2.9695e-04
Epoch 3/5
461/819 [===============>..............] - ETA: 14s - loss: nan - accuracy: 3.0504e-04

KeyboardInterrupt: ignored

lstm layers

In [ ]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.LSTM(16)(inputs)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)
callbacks = [
keras.callbacks.ModelCheckpoint("jena_lstm.keras",
save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
epochs=10,
validation_data=val_dataset,
callbacks=callbacks)

Epoch 1/10
819/819 [==============================] - 79s 94ms/step - loss: 47.1614 - mae: 5.0273 - val_loss: 13.3742 - val_mae: 2.7676
Epoch 2/10
819/819 [==============================] - 76s 93ms/step - loss: 11.3946 - mae: 2.6005 - val_loss: 9.6408 - val_mae: 2.4195
Epoch 3/10
819/819 [==============================] - 76s 93ms/step - loss: 9.9799 - mae: 2.4521 - val_loss: 10.0972 - val_mae: 2.4635
Epoch 4/10
819/819 [==============================] - 77s 94ms/step - loss: 9.5472 - mae: 2.4040 - val_loss: 9.9247 - val_mae: 2.4364
Epoch 5/10
819/819 [==============================] - 78s 95ms/step - loss: 9.2360 - mae: 2.3633 - val_loss: 9.9039 - val_mae: 2.4257
Epoch 6/10
819/819 [==============================] - 77s 94ms/step - loss: 9.0372 - mae: 2.3357 - val_loss: 9.9751 - val_mae: 2.4361
Epoch 7/10
819/819 [==============================] - 78s 94ms/step - loss: 8.8951 - mae: 2.3145 - val_loss: 10.0023 - val_mae: 2.4446
Epoch 8/10
819/819 [==============================] - 78s

In [ ]:
model = keras.models.load_model("jena_lstm.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

405/405 [==============================] - 21s 50ms/step - loss: 10.9631 - mae: 2.5933
Test MAE: 2.59


lstm with drop out 

In [ ]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.LSTM(32, recurrent_dropout=0.25)(inputs)
x = layers.LayerNormalization()(x)
x = layers.LSTM(64, recurrent_dropout=0.25)(inputs)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(1)(x)

model = keras.Model(inputs, outputs)
model.summary()
callbacks = [
keras.callbacks.ModelCheckpoint("jena_lstm_dropout.keras",
save_best_only=True)
]


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 120, 14)]         0         
                                                                 
 lstm_2 (LSTM)               (None, 64)                20224     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 20,289
Trainable params: 20,289
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer="rmsprop", loss="mse", metrics=["accuracy"])
history = model.fit(train_dataset,
epochs=5,
validation_data=val_dataset,
callbacks=callbacks)

GRU

In [ ]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.GRU(32, recurrent_dropout=0.5, return_sequences=True)(inputs)
x = layers.GRU(32, recurrent_dropout=0.5)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)
callbacks = [
keras.callbacks.ModelCheckpoint("jena_stacked_gru_dropout.keras",
save_best_only=True)
]


model.compile(optimizer="rmsprop", loss="mse", metrics=["accuracy"])
history = model.fit(train_dataset,
epochs=50,
validation_data=val_dataset,
callbacks=callbacks)
model = keras.models.load_model("jena_stacked_gru_dropout.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

In [ ]:
from tensorflow.keras.layers.experimental import LayerNormalization
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.LSTM(32, recurrent_dropout=0.25)(inputs)
x=LayerNormalization(axis=-1 , center=True , scale=True)(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1)(x)

model = keras.Model(inputs, outputs)
callbacks = [
keras.callbacks.ModelCheckpoint("jena_lstm_dropout.keras",
save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["accuracy"])
history = model.fit(train_dataset,
epochs=5,
validation_data=val_dataset,
callbacks=callbacks)
